In [1]:
import pandas as pd 
import xml.etree.ElementTree as et 
import os
import regex as re
import json
import os
from tqdm.autonotebook import tqdm

/Users/max/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [43]:
columns = ['filename', 'rootelem', 'rootid', 'rootname', 'path', 'key', 'value', 'name', 'id', 'type', \
           'sourceref', 'targetref', 'processref', 'bpmnelement', 'x', 'y', 'width', 'height', 'bpmntype']
print(len(columns))
df = pd.DataFrame(columns=columns)
df.head()

19


,filename,rootelem,rootid,rootname,path,key,value,name,id,type,sourceref,targetref,processref,bpmnelement,x,y,width,height,bpmntype


In [44]:
def parse_bpmn_XML(xml_file): 
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    return iterxml(xroot, xml_file)
    
    
def iterxml(xml, filename):
    df = pd.DataFrame(columns=columns)
    countproperty = 0
    for elem in list(xml):
        path = ""
        firsttabs = ""
        rootelem = ""
        if "}" in elem.tag:
            rootelem = elem.tag.split("}", 1)[1]
        else:
            rootelem = elem.tag
            
#         print(firsttabs + rootelem)
#         print(firsttabs + "  " + str(elem.attrib))
        rootid = elem.attrib.get("id")
        rootname = elem.attrib.get("name")
        path += "/"+rootelem
        rootpath1 = path
        for items in elem:
            path = rootpath1
            firsttabs = "\t"
            tagname = ""
            if "}" in items.tag:
                tagname = items.tag.split("}", 1)[1]
            else:
                tagname = items.tag
                
            path += "/"+tagname
#             print(firsttabs + tagname)
#             print(firsttabs + "1  " + str(items.attrib))
            elemid = items.attrib.get("id", None)
            elemname = items.attrib.get("name", None)
            elemtype = items.attrib.get("{http:///com.sap.ifl.model/Ifl.xsd}type", None)
            elemsource = items.attrib.get("sourceRef", None)
            elemtarget = items.attrib.get("targetRef", None)
            elemprocess = items.attrib.get("processRef", None)
            elembpmnelement = items.attrib.get("bpmnElement", None)
            if len(list(items)) == 0:
                df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                       'rootname':rootname, 'path':path, 'key':None, 'value':None, \
                       'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                       'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                       'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                          ignore_index=True)
            elif tagname == "extensionElements":
                rootpath2 = path
                for prop in items:
#                     path = rootpath1
                    key = None
                    val = None
                    tagname = ""
                    if "}" in prop.tag:
                        tagname = prop.tag.split("}", 1)[1]
                    else:
                        tagname = prop.tag
#                     path = rootpath2+"/"+tagname
                    for keyval in prop:
                        tagname = ""
                        if "}" in keyval.tag:
                            tagname = keyval.tag.split("}", 1)[1]
                        else:
                            tagname = keyval.tag
#                         print(tagname)
#                         print(keyval.text)
                        if tagname == "key":
                            key = keyval.text
                        elif tagname == "value":
                            val = keyval.text   
                    df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                               'rootname':rootname, 'path':path, 'key':key, 'value':val, \
                               'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                               'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                               'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                      ignore_index=True)
            elif tagname == "conditionExpression":
                df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                               'rootname':rootname, 'path':path, 'key':tagname, 'value':items.text, \
                               'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                               'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                               'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                ignore_index=True)
            elif tagname == "outgoing": 
                df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                               'rootname':rootname, 'path':path, 'key':tagname, 'value':items.text, \
                               'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                               'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                               'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                ignore_index=True)
            elif tagname == "incoming": 
                df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                               'rootname':rootname, 'path':path, 'key':tagname, 'value':items.text, \
                               'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                               'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':None,\
                               'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                ignore_index=True)
            else:
                df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                       'rootname':rootname, 'path':path, 'key':None, 'value':None, \
                       'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                       'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                       'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                          ignore_index=True)
                firsttabs += "\t"
                rootpath2 = rootpath1 + "/" + tagname
                for case in items:
                    path = rootpath2
                    firsttabs = "\t\t"
                    tagname = ""
                    if "}" in case.tag:
                        tagname = case.tag.split("}", 1)[1]
                    else:
                        tagname = case.tag
#                     print(case.attrib)
                    path = rootpath2+"/"+tagname
                    elemid = case.attrib.get("id", None)
                    elemname = case.attrib.get("name", None)
                    elemtype = case.attrib.get("{http:///com.sap.ifl.model/Ifl.xsd}type", None)
                    elemsource = case.attrib.get("sourceRef", None)
                    elemtarget = case.attrib.get("targetRef", None)
                    elemprocess = case.attrib.get("processRef", None)
                    elembpmnelement = case.attrib.get("bpmnElement", None)
                    firsttabs += "\t"
                    if tagname == "property":
                        countproperty+=1
                    if len(list(case)) == 0:
                        df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                               'rootname':rootname, 'path':path, 'key':None, 'value':None, \
                               'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                               'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                               'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                  ignore_index=True)
                    elif tagname == "extensionElements":
                        for prop in case:
                            tagname = ""
                            if "}" in prop.tag:
                                tagname = prop.tag.split("}", 1)[1]
                            else:
                                tagname = prop.tag

#                             path += "/"+tagname
                            key = None
                            val = None
                            for keyval in prop:
                                tagname = ""
                                if "}" in keyval.tag:
                                    tagname = keyval.tag.split("}", 1)[1]
                                else:
                                    tagname = keyval.tag
                                if tagname == "key":
                                    key = keyval.text
                                elif tagname == "value":
                                    val = keyval.text  
                            df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                   'rootname':rootname, 'path':path, 'key':key, 'value':val, \
                                   'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                                   'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                                   'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                      ignore_index=True)
                    elif tagname == "conditionExpression": 
                        df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                       'rootname':rootname, 'path':path, 'key':tagname, 'value':case.text, \
                                       'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                                       'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                                       'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                              ignore_index=True)
                    elif tagname == "outgoing": 
                        df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                       'rootname':rootname, 'path':path, 'key':tagname, 'value':case.text, \
                                       'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                                       'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                                       'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                              ignore_index=True)
                    elif tagname == "incoming": 
                        df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                       'rootname':rootname, 'path':path, 'key':tagname, 'value':case.text, \
                                       'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                                       'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                                       'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                              ignore_index=True)
                    elif tagname == "BPMNShape":
                        for local in case:
                            height = local.attrib.get("height", None)
                            width = local.attrib.get("width", None)
                            x = local.attrib.get("x", None)
                            y = local.attrib.get("y", None)
                            df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                   'rootname':rootname, 'path':path, 'key':None, 'value':None, \
                                   'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                                   'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                                   'x':x, 'y':y, 'width':width, 'height':height, 'bpmntype':None},\
                                      ignore_index=True)
                    elif tagname == "BPMNEdge":
                        for local in case:
                            localtype = local.attrib.get("{http://www.w3.org/2001/XMLSchema-instance}type", None)
                            x = local.attrib.get("x", None)
                            y = local.attrib.get("y", None)
                            df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                   'rootname':rootname, 'path':path, 'key':None, 'value':None, \
                                   'name':elemname, 'id':elemid, 'type':elemtype, 'sourceref':elemsource,\
                                   'targetref':elemtarget, 'processref':elemprocess, 'bpmnelement':elembpmnelement,\
                                   'x':x, 'y':y, 'width':None, 'height':None, 'bpmntype':localtype},\
                                      ignore_index=True)
                            
                    else:
                        rootpath3 = path
                        for prop in case:
                            path = rootpath3
                            tagname = ""
                            if "}" in prop.tag:
                                tagname = prop.tag.split("}", 1)[1]
                            else:
                                tagname = prop.tag
                            
                            path += "/"+tagname
                            if tagname == "property":
                                countproperty+=1
                            localelemid = prop.attrib.get("id", None)
                            localelemname = prop.attrib.get("name", None)
                            localelemtype = prop.attrib.get("{http:///com.sap.ifl.model/Ifl.xsd}type", None)
                            localelemsource = prop.attrib.get("sourceRef", None)
                            localelemtarget = prop.attrib.get("targetRef", None)
                            localelemprocess = prop.attrib.get("processRef", None)
                            localelembpmnelement = prop.attrib.get("bpmnElement", None)
                            if tagname == "extensionElements":
                                for local in prop:
                                    tagname = ""
                                    if "}" in local.tag:
                                        tagname = local.tag.split("}", 1)[1]
                                    else:
                                        tagname = local.tag
                                    key = None
                                    val = None
                                    for keyval in local:
                                        tagname = ""
                                        if "}" in keyval.tag:
                                            tagname = keyval.tag.split("}", 1)[1]
                                        else:
                                            tagname = keyval.tag
                                        if tagname == "key":
                                            key = keyval.text
                                        elif tagname == "value":
                                            val = keyval.text  
                                    df = df.append({'filename':filename, 'rootelem':rootelem, 'rootid':rootid, \
                                           'rootname':rootname, 'path':path, 'key':key, 'value':val, \
                                           'name':localelemname, 'id':localelemid, 'type':localelemtype, 'sourceref':localelemsource,\
                                           'targetref':localelemtarget, 'processref':localelemprocess, 'bpmnelement':localelembpmnelement,\
                                           'x':None, 'y':None, 'width':None, 'height':None, 'bpmntype':None},\
                                              ignore_index=True)
                            

    return df

In [204]:
file = "BYD0550_CustomerInvoice_ByD_to_BDS.iflw"
df = parse_bpmn_XML(file)

In [53]:
df.info(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37601 entries, 0 to 97
Data columns (total 19 columns):
filename       37601 non-null object
rootelem       37601 non-null object
rootid         37442 non-null object
rootname       35676 non-null object
path           37601 non-null object
key            21159 non-null object
value          18073 non-null object
name           2933 non-null object
id             12921 non-null object
type           612 non-null object
sourceref      1924 non-null object
targetref      1924 non-null object
processref     253 non-null object
bpmnelement    8366 non-null object
x              6951 non-null object
y              6951 non-null object
width          2424 non-null object
height         2424 non-null object
bpmntype       4527 non-null object
dtypes: object(19)
memory usage: 5.7+ MB


In [52]:
df.head(20)

,filename,rootelem,rootid,rootname,path,key,value,name,id,type,sourceref,targetref,processref,bpmnelement,x,y,width,height,bpmntype
0,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,namespaceMapping,None,None,None,None,None,None,None,None,None,None,None,None,None
1,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,allowedHeaderList,None,None,None,None,None,None,None,None,None,None,None,None,None
2,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,httpSessionHandling,None,None,None,None,None,None,None,None,None,None,None,None,None
3,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,ServerTrace,false,None,None,None,None,None,None,None,None,None,None,None,None
4,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,returnExceptionToSender,false,None,None,None,None,None,None,None,None,None,None,None,None
5,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,log,All events,None,None,None,None,None,None,None,None,None,None,None,None
6,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,componentVersion,1.1,None,None,None,None,None,None,None,None,None,None,None,None
7,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,cmdVariantUri,ctype::IFlowVariant/cname::IFlowConfiguration/...,None,None,None,None,None,None,None,None,None,None,None,None
8,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/participant,None,None,Sender,Participant_1,EndpointSender,None,None,None,None,None,None,None,None,None
9,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/participant/extensionElements,enableBasicAuthentication,false,None,None,None,None,None,None,None,None,None,None,None,None


In [54]:
df.to_csv("result.csv")

In [45]:
import glob

In [62]:
import os
iflowfiles = []
iflowdirs = []
# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in os.walk("."):
    path = root.split(os.sep)
    foundfiles = glob.glob(root+'/*.iflw')
#     if len(path) >= 2:
#         for file in foundfiles:
#             iflowdirs = iflowdirs + list(path[1])
            
#     print(path[1])
#     print(root)
#     print((len(path) - 1) * '---', os.path.basename(root))
#     for file in files:
    
        
    iflowfiles = iflowfiles + foundfiles

In [63]:
iflowinfo = zip(iflowdirs,iflowfiles)

In [64]:
for i in iflowinfo:
    print(i[0],i[1])

In [58]:
len(iflowfiles)

140

In [47]:
files = iflowfiles
dfset = []
pbar = tqdm(total=len(iflowfiles), leave=False)
pbar.set_description(desc='Processing iflows', refresh=True)

for i in files: 
    df = parse_bpmn_XML(i)
    pbar.update(1)
#     print(len(df.columns))
#     df.dropna(axis=0, how='all', subset=df.columns.values, inplace=True)
    dfset.append(df)
pbar.close()

In [48]:
df = dfset[0]
pbar = tqdm(total=len(dfset), leave=False)
pbar.set_description(desc='Processing iflows', refresh=True)
pbar.update(1)
for i in range(1, len(dfset)):
    df = df.append(dfset[i])
    pbar.update(1)
pbar.close()
df.to_csv("iflows.csv")

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37601 entries, 0 to 97
Data columns (total 19 columns):
filename       37601 non-null object
rootelem       37601 non-null object
rootid         37442 non-null object
rootname       35676 non-null object
path           37601 non-null object
key            21159 non-null object
value          18073 non-null object
name           2933 non-null object
id             12921 non-null object
type           612 non-null object
sourceref      1924 non-null object
targetref      1924 non-null object
processref     253 non-null object
bpmnelement    8366 non-null object
x              6951 non-null object
y              6951 non-null object
width          2424 non-null object
height         2424 non-null object
bpmntype       4527 non-null object
dtypes: object(19)
memory usage: 5.7+ MB


In [51]:
len(df['filename'].unique())

140

In [7]:
df = pd.read_csv("iflows.csv")

In [12]:
names = df['filename'].unique()
filenamemapp = {}
for i in names:
    for j in reversedproperties:
        if j in i:
            filenamemapp.update({i:reversedproperties.get(j)})
df['package'] = df['filename'].map(filenamemapp)

In [13]:
df.head()

,Unnamed: 0,filename,rootelem,rootid,rootname,path,key,value,name,id,...,sourceref,targetref,processref,bpmnelement,x,y,width,height,bpmntype,package
0,0,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,namespaceMapping,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,testmatthijs
1,1,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,allowedHeaderList,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,testmatthijs
2,2,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,httpSessionHandling,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,testmatthijs
3,3,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,ServerTrace,false,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,testmatthijs
4,4,./CPI Basis Oefening 14.iflw,collaboration,Collaboration_1,Default Collaboration,/collaboration/extensionElements,returnExceptionToSender,false,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,testmatthijs


In [4]:
teststring = "./teststring"
test = teststring.split("/")
print(test)

['.', 'teststring']


In [74]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

In [69]:
df_with_dummies = pd.get_dummies(df)

In [76]:
df_with_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37601 entries, 0 to 97
Columns: 10714 entries, filename_./API_MATERIAL/src/main/resources/scenarioflows/integrationflow/API_MATERIAL.iflw to bpmntype_dc:Point
dtypes: uint8(10714)
memory usage: 384.5 MB


In [70]:
X = df_with_dummies.to_numpy()

In [71]:
X[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [73]:
# manually fit on batches
kmeans = MiniBatchKMeans(n_clusters=2,
                         random_state=0,
                         batch_size=6)
kmeans = kmeans.partial_fit(X[0:6,:])
kmeans = kmeans.partial_fit(X[6:12,:])
kmeans.cluster_centers_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [78]:
kmeans = KMeans(n_clusters=10, random_state=0, n_jobs=-1).fit(X)
kmeans.labels_

array([6, 6, 6, ..., 7, 7, 7], dtype=int32)

In [79]:
len(kmeans.labels_)

37601

In [ ]:
import seaborn as sns
sns.set(style="ticks", color_codes=True)

g = sns.pairplot(df_with_dummies)